# Memory

In [1]:
from dotenv import load_dotenv # dotenv 파일을 읽어 환경변수로 설정
load_dotenv()

True

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [3]:
store = {}  # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [4]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you']

In [5]:
# LangChain의 OpenAI API를 사용하기 위한 클래스
from langchain_openai import ChatOpenAI
# LangChain의 프롬프트 템플릿을 사용하기 위한 클래스
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 메시지 기록을 사용하는 Runnable 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory

# LangChain의 메시지 템플릿을 사용하기 위한 클래스
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}를 잘하는 AI 어시스턴트야.'),   # 시스템 메시지
    MessagesPlaceholder(variable_name="history"),  # 메시지 기록을 위한 플레이스홀더
    ('human', '{query}'),  # 사용자의 질문을 위한 플레이스홀더
])

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
chain = prompt | model


In [11]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [12]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 농장에서 나무를 세 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 농장에서 나무를 세 그루 키우고 있다니, 정말 귀엽고 재미있는 설정이네요! 어떤 종류의 나무를 키우고 있는지, 혹은 그 나무들에 대해 어떤 이야기가 있는지 궁금합니다. 또는 토끼가 나무를 키우는 이유에 대해서도 이야기해볼까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 41, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BOi9FD0x8Gkr4BEOxzeGX4jGrY7QO', 'finish_reason': 'stop', 'logprobs': None} id='run-332d74da-da2d-4c1f-abb1-462916c9e488-0' usage_metadata={'input_tokens': 41, 'output_tokens': 77, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [13]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니, 정말 귀엽고 재미있는 설정이네요! 어떤 종류의 나무를 키우고 있는지, 혹은 그 나무들에 대해 어떤 이야기가 있는지 궁금합니다. 또는 토끼가 나무를 키우는 이유에 대해서도 이야기해볼까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 41, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BOi9FD0x8Gkr4BEOxzeGX4jGrY7QO', 'finish_reason': 'stop', 'logprobs': None}, id='run-332d74da-da2d-4c1f-abb1-462916c9e488-0', usage_metadata={'input_tokens': 41, 'output_tokens': 77, 'total_tokens': 118, 'input_token_detai

In [14]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼와 다람쥐는 합해서 몇 그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 농장에서 나무를 세 그루 키우고 있다고 하셨으니, 다람쥐가 키우는 나무의 수를 알려주시면 합쳐서 몇 그루의 나무를 키우고 있는지 계산할 수 있을 것 같습니다. 다람쥐가 키우는 나무의 수는 몇 그루인가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 149, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BOi9cItIZSGWuVRdXt21CyVLjARVO', 'finish_reason': 'stop', 'logprobs': None} id='run-e71c7072-e350-4c96-b6dd-a2b582b60abc-0' usage_metadata={'input_tokens': 149, 'output_tokens': 80, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
